<a href="https://colab.research.google.com/github/lordWalnuts/jupyter-notebooks/blob/main/peaky_blinders_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up

In [ ]:
!pip install -Uqq fastai
!pip install duckduckgo_search

# Collecting the Data

### Downloading the Data

In [ ]:
from duckduckgo_search import ddg_images
from fastcore.all import *
from fastdownload import download_url
from fastai.vision.all import *


In [ ]:
def search_images(term,max_images=30):
    print(f"Searching for '{term}'")
    return L(ddg_images(term,max_results=max_images)).itemgot('image')

In [ ]:

search_terms_1 = ['Thomas Shelby','Arthur Shelby','Ada Shelby','Polly Gray','Michael Gray','John Shelby']
search_terms_2 = ['Cillian Murphy','Paul Anderson','Sophie Rundle','Helen McCrory','Joe Cole','Finn Cole']


path = Path('peaky_blinders_family')
from time import sleep

for index in range(0,len(search_terms_1)):
  dest = (path/search_terms_1[index])
  dest.mkdir(exist_ok=True,parents=True)
  download_images(dest,urls=search_images(f'{search_terms_1[index]} peaky blinders photo',max_images=30))
  sleep(10)
  download_images(dest,urls=search_images(f'{search_terms_2[index]} actor photo',max_images=30))
  resize_images(path/search_terms_1[index],max_size=400,dest=path/search_terms_1[index])

### Cleaning the Data

In [ ]:
path = Path('peaky_blinders_family')
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

6

### Moving the dataset to Google Drive(Optional)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp peaky_blinders_family/ drive/MyDrive/dataset/ -r

# Loading the data into Dataloaders

In [ ]:
dls = DataBlock(
    blocks = (ImageBlock,CategoryBlock),
    get_items= get_image_files,
    splitter = RandomSplitter(valid_pct=0.2,seed=42),
    get_y = parent_label,
    item_tfms = [Resize(192,method="squish")]
)
dls = dls.dataloaders(path,bs=32)

# Training the Model

In [ ]:
learn = vision_learner(dls,resnet50,metrics=error_rate)
learn.fine_tune(5)

epoch,train_loss,valid_loss,error_rate,time
0,2.344597,1.069694,0.426471,00:02


epoch,train_loss,valid_loss,error_rate,time
0,0.818901,0.911779,0.308824,00:02
1,0.502134,1.006084,0.338235,00:03
2,0.375839,0.983983,0.279412,00:03
3,0.288586,0.886793,0.279412,00:03
4,0.235215,0.843834,0.279412,00:02


# Testing our Model

In [ ]:
def changeName(name):
     if name =="Cillian Murphy":
        return "Thomas Shelby"
     elif name == "Finn Cole":
        return "Michael Gray"
     elif name == "Helen McCrory":
        return "Thomas Gray"
     elif name == "Joe Cole":
        return "John Shelby"
     elif name == "Paul Anderson":
        return "Arthur Shelby"
     else:
        return "Ada Shelby"
     

In [ ]:
im = Path('drive/MyDrive/peaky_test/art1.jpg')
member,_,probs = learn.predict(PILImage.create(im))
print(f"This is {member}")
print(f"Probability it's {member}: {probs[0]:.4f}")

This is Arthur Shelby
Probability it's Arthur Shelby: 0.0000
